In [1]:
import boto
import dask.dataframe as dd
import pandas as pd
import os
import re
import logging
from dask.delayed import delayed
from distributed import Client
from functools import partial

In [2]:
s3 = boto.connect_s3()
bucket = s3.get_bucket('data.mindhypertrophy.com')

In [4]:
client = Client('10.0.2.76:8786')

distributed.comm.core - DEBUG - sleeping on connect
distributed.comm.core - DEBUG - sleeping on connect
distributed.comm.core - DEBUG - sleeping on connect
distributed.comm.core - DEBUG - sleeping on connect
distributed.utils - ERROR - Timed out trying to connect to 'tcp://10.0.2.76:8786' after 5 s: ConnectionRefusedError: [Errno 10061] Unknown error
Traceback (most recent call last):
  File "C:\Users\Andrew Meier\Miniconda2\envs\py3\lib\site-packages\distributed\comm\core.py", line 159, in connect
    quiet_exceptions=EnvironmentError)
  File "C:\Users\Andrew Meier\Miniconda2\envs\py3\lib\site-packages\tornado\gen.py", line 1055, in run
    value = future.result()
  File "C:\Users\Andrew Meier\Miniconda2\envs\py3\lib\site-packages\tornado\concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
tornado.gen.TimeoutError: Timeout

During handling of the above exception, another exception occurred:

Traceback (most recent call la

OSError: Timed out trying to connect to 'tcp://10.0.2.76:8786' after 5 s: ConnectionRefusedError: [Errno 10061] Unknown error

In [ ]:
# match all the loan tapes from the year 2011
pattern = re.compile('\d{8}/loan_stats.csv')
host = f's3://{bucket.name}'
urls = [key.generate_url(expires_in=600) for key in bucket.list() if pattern.search(key.name)]
print(f'found {len(urls)} urls')

In [ ]:
def read_csv_async(url):
    print(f'reading {url}')
    df = pd.read_csv(url, index_col=False)
    return df

In [ ]:
# create a list of 'future' data frame objects
future_frames = [delayed(read_csv_async)(url) for url in urls]
# create a 'future' DataFrame object
df_future = dd.from_delayed(future_frames)

In [ ]:
%%time
df = df_future.compute()

In [ ]:
df.head()